In [1]:
import requests
import datetime
from datetime import datetime
import wget
import logging

In [2]:
logging.basicConfig(level=logging.INFO, filename="pc.log", format="%(asctime)s - %(levelname)s - %(message)s")

In [3]:
#convertendo as horas do documento .txt no formato específico

from datetime import datetime

lf = list(open('/home/jose/IC/pc/datas_relevantes.txt','r'))
lf1 = [s.rstrip() for s in lf]
lf2 = [s.replace(" ", "") for s in lf1]
lf3 = [datetime.strptime(s, '%d/%m/%Y').date() for s in lf2]
datas = [i.strftime('%Y%m%d') for i in lf3]

In [4]:
#construindo horas do dia:

r1 = [str(r) for r in range(24)]
r2 = [i.zfill(2) for i in r1]

In [5]:
#juntando as datas com suas respectivas 23 horas:

datas_horas = [j + i for j in datas for i in r2]

In [6]:
logging.info("Iniciando requisição")
requisicao = [requests.get('https://api-redemet.decea.mil.br/produtos/radar/03km?api_key=P53UkUuaqdJ3GjMoRhNElsIf5gzBD2w2iwPNqhUd&radar=pc&anima=3&data=' + i) for i in datas_horas]
logging.info("Requisição completa")

In [7]:
#transformando objeto retornado em json:

json = [i.json() for i in requisicao]

In [8]:
true = [i for i in json if i["status"] ==True]

In [9]:
#"quebrando" as primeiras chaves do arquivo:

radares = [i["data"]["radar"] for i in true]

A API organiza a função "anima" criando listas com a quantidade inserida pelo usuário. Como nosso anima = 3, serão três listas:

In [10]:
#acessando as respectivas listas que a função "anima" retornou:

rad1 = [i[0] for i in radares]
rad2 = [i[1] for i in radares]
rad3 = [i[2] for i in radares]
radares_1 = rad1 + rad2 + rad3

In [23]:
nome = [i[11] for i in radares_1]
nome_t = [x for x in nome if x["nome"] != "Radar - Pico do Couto/RJ"]

In [12]:
date1 = [i[11]["data"] for i in radares_1]
date_t = [x for x in date1 if x is not None]
date_t = [datetime.strptime(i, '%Y-%m-%d %H:%M:%S').date() for i in date_t]
date_days = [i.strftime('%Y%m%d') for i in date_t]
date_days = list(set(date_days))

In [20]:
with open('DATAS_COM_IMAGENS.txt', 'w') as arquivo:
    arquivo.write(str(date_days))

In [21]:
len(date_days)

90

In [13]:
datas_sem_retorno = list(set(datas) - set(date_days))
print(datas_sem_retorno[:5]) 
logging.warning(f'Datas sem retorno da api, ou sem dados válidos: {datas_sem_retorno}')

['20180125', '20161220', '20191211', '20191212', '20190407']


In [15]:
#Acessando os paths do radar pico do couto:

pc = [i[11]["path"] for i in radares_1]
path_pc = list(set(pc))
path_pc = [i for i in path_pc if type(i) is str]

In [16]:
#Baixando imagens no repositório no qual esse notebook foi aberto:

for i in path_pc:

 image_url = i
 image_filename = wget.download(image_url)

logging.info("Imagens baixadas")

100% [..............................................................] 451 / 451